## Collaborative Filtering

In [3]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [139]:
book_ratings = pd.read_csv('~/Downloads/BX-CSV-Dump/BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('~/Downloads/BX-CSV-Dump/BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\DPS\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:

* Explore both datasets

In [142]:
book_ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [143]:
book_ratings.describe()

,User-ID,Book-Rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [144]:
book_ratings.shape

(1149780, 3)

In [151]:
book_ratings=book_ratings.head(20000)

In [152]:
book_ratings.shape

(20000, 3)

In [153]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [154]:
books.describe()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
count,271360,271360,271359,271360,271358,271360,271360,271357
unique,271360,242135,102023,202,16807,271044,271044,271041
top,0619000279,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/042519597X.0...,http://images.amazon.com/images/P/006091291X.0...,http://images.amazon.com/images/P/033028407X.0...
freq,1,27,632,13903,7535,2,2,2


In [155]:
books.shape

(271360, 8)

* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [156]:
df_book_features = pd.pivot_table(book_ratings, values='Book-Rating', index='ISBN', columns='User-ID').fillna(0)

In [157]:
df_book_features

User-ID,2,7,8,9,10,12,14,16,17,19,...,278832,278836,278838,278843,278844,278846,278849,278851,278852,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
0002005018,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002231115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002232766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002240114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000225669X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N3453124715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NONFICTION,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
O6712345670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* create the instance of the NearestNeighbors class

In [158]:
knn = NearestNeighbors(n_neighbors=5, algorithm='auto')

* fit the NearestNeighbors using'df_book_features'

In [159]:
knn.fit(df_book_features)
k=5
new_data = df_book_features.loc['0006511929'].values.reshape(1, -1)

['0453007481', '0892967994', '0453008739', '0460012231', '0893752649']

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [274]:
def recommendation(Book_title):
    print("Here are the top 5 recommendations corresponding to the book '"+Book_title+"':")
    isbn = books.loc[books['Book-Title'] == Book_title, 'ISBN'].iloc[0]
    new_data = df_book_features.loc[isbn].values.reshape(1, -1)
    distances, indices = knn.kneighbors(new_data, n_neighbors=7)
    similar_isbn = df_book_features.iloc[indices[0][1:]].index.tolist()
   # similar_isbn = [i.strip("'") for i in similar_isbn if i.strip("'") != isbn]
    if isbn in similar_isbn:
        similar_isbn.remove(isbn)
    for i in similar_isbn:
        try:
            title = books.loc[books['ISBN'] == i, 'Book-Title'].values[0]
            distance = distances[0][similar_isbn.index(i)]
            print(title + " (distance: " + str(distance) + ")")
        except IndexError:
            pass

* Apply the function to book of your choice

In [275]:
recommendation('Clara Callan')

Here are the top 5 recommendations corresponding to the book 'Clara Callan':
The Middle Stories (distance: 0.0)
Jane Doe (distance: 0.0)
More Cunning Than Man: A Social History of Rats and Man (distance: 0.0)
The Witchfinder (Amos Walker Mystery Series) (distance: 0.0)
Goodbye to the Buttermilk Sky (distance: 1.0)


In [276]:
recommendation('Decision in Normandy')

Here are the top 5 recommendations corresponding to the book 'Decision in Normandy':
German Boy: A Child in War (distance: 0.0)
Currahee!:  A Screaming Eagle at Normandy (distance: 0.0)
Our Kind of People: Inside America's Black Upper Class (distance: 0.0)
American Chica: Two Worlds, One Childhood (distance: 1.0)
War's End: An Eyewitness Account of America's Last Atomic Mission (distance: 1.0)
